In [1]:
import depthquality.quality as quality
import depthquality.meshes as meshes
import pkg_resources
import os

/home/mprat/Documents/repos/depth-quality/src/depthquality/transformations.py:1926: UserWarning: No module named 'depthquality._transformations'
  warnings.warn(str(err))


A sample of running the horizontal cylinder code through the pipeline, and visualizing it with Meshcat.

In [2]:
folder_name = "vert_cylinders"
rgb_filename = os.path.join("..", "src", "tests", "data", folder_name, "1.png")
pc_filename = os.path.join("..", "src", "tests", "data", folder_name, "1.dat")
pointcloud_filename = os.path.join("..", "src", "tests", "data", folder_name, "1.ply")
reference_mesh = meshes.VERTICAL_CYLINDERS

In [3]:
aligned_pointcloud, camera_angle = quality.align_pointcloud_to_reference(
    reference_mesh, rgb_filename, pc_filename, pointcloud_filename, depth_scale=0.001)

# if you want to save the pointcloud to disk and load it in another visualizer
# quality.save_pointcloud(pointcloud_filename, "transformed", aligned_pointcloud)

cropped_pointcloud = quality.clip_pointcloud_to_pattern_area(
    reference_mesh, aligned_pointcloud, depth_scale=0.001)

# if you want to save the pointcloud to disk and load it in another visualizer
# quality.save_pointcloud(pointcloud_filename, "cropped", cropped_pointcloud)

rmse, density = quality.calculate_rmse_and_density(
    ground_truth_mesh=reference_mesh,
    cropped_pointcloud=cropped_pointcloud,
    depth_scale=0.001,
    camera_angle=camera_angle)

In [4]:
print("RMSE = {}, density= {}".format(rmse, density))
print(camera_angle)

RMSE = 1.676035779349817, density= 1.765164458211962
[ 0.00550642 -0.11431424  0.99342938]


We can use Meshcat to visualize our geometry directly in a Jupyter Notebook.

In [5]:
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tfms
import numpy as np

In [6]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [7]:
vis['reference'].set_object(g.ObjMeshGeometry.from_file(reference_mesh.path))
vis['reference'].set_transform(tfms.scale_matrix(0.001))

In [9]:
vis['transformed_cloud'].set_object(
    g.PointCloud(np.asarray(aligned_pointcloud.points).T,
                          np.asarray(aligned_pointcloud.colors).T)
)

vis['cropped_cloud'].set_object(
    g.PointCloud(np.asarray(cropped_pointcloud.points).T,
                          np.asarray(cropped_pointcloud.colors).T)
)